In [3]:
from bs4 import BeautifulSoup
import requests
import re
from datetime import date
from datetime import datetime, timedelta
import json 
import locale
import csv
import pandas as pd
from csv import DictWriter
from collections import defaultdict

In [2]:
front_page = 'https://www.estrepublicain.fr/sante/coronavirus'

In [3]:
def fetch_links(source_website):
    """this function fetches links from the front page
    and returns a list"""
    
    list_of_links = []
    
    #pagination on the website follows a simple pattern
    #'https://www.estrepublicain.fr/sante/coronavirus?page=NUMBER_OF_PAGE
    #thus, we run a loop 
    
    for i in range(101): 
        url = source_website + '?page=' + str(i)
        page = requests.get(url)    
        data = page.text
        soup = BeautifulSoup(data)
        
        #catching bugs if a link doesn't contain a substring
        for link in soup.find_all('a'):
            try:
                if link.get('href').startswith('/sante/2'):
                    list_of_links.append('https://www.estrepublicain.fr'+link.get('href'))
            except AttributeError:
                continue
    return list_of_links

In [309]:
big_list = fetch_links('https://www.estrepublicain.fr/sante/coronavirus')

In [328]:
big_list[0]

'https://www.estrepublicain.fr/sante/2021/10/14/covid-19-le-pass-sanitaire-ne-sera-pas-eternel-mais-il-va-etre-prolonge'

In [ ]:
with open('list_of_links.txt', 'w') as f:
    for item in big_list:
        f.write("%s\n" % item)

In [20]:
def grab_info(link):
    """this function scrapes information
    from a single web page, including text, title, and date"""
    
    #headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }
    
    
    page_single = requests.get(link).text
    soup_single = BeautifulSoup(page_single)
    
    #check if the article is not behind a paywall
    if soup_single.find_all('div', class_='preview non-paywall2'):
        return None
    
    #fetching the date from the link string
    date_from_link = link.split('/sante/')[1].split('/')
    date_fin = '/'.join(date_from_link[:3])
    
    #putting together all the text from the page
    text = []
    for i in soup_single.find_all('p', class_=None)[8:]:
        text.append(i.get_text())
    text = text[1:]
    text = ' '.join(text)
    text = text
    
    #fetching the title
    #two options depending on the page structure
    try:
        title = soup_single.find('title').get_text()
    except:
        title = soup_single.find("span",{"class":"headline"}).next_sibling.lstrip()
    
    #getting reference summaries
    meta_tag = soup_single.find('meta', attrs={'name':'Description'})
    summary = meta_tag['content']
    
    
    dictionary = {'date': date_fin,
               'summary':summary,
               'text':text,
               'title':title,
               'topic':'',
               'url': link}
    return dictionary

#json.dumps(dictionary, indent = 6, ensure_ascii=False)

In [21]:
grab_info('https://www.estrepublicain.fr/sante/2021/11/17/covid-19-les-contaminations-en-forte-hausse-espoir-autour-du-traitement-de-pfizer')

{'date': '2021/11/17',
 'summary': "Suivez avec nous l'évolution de la pandémie de Covid-19 en France et dans le monde ce mercredi 17 novembre.",
 'text': '"On est tous humains" : le porte-parole du gouvernement Gabriel Attal a défendu le Premier ministre Jean Castex et le ministre de l\'Intérieur Gérald Darmanin, critiqués à la suite d\'une vidéo tournée mardi soir les montrant serrant des mains, sans masque, au mépris des gestes barrières.\xa0Gabriel Attal a\xa0concédé qu\'"il peut y avoir de temps en temps un moment d\'inattention, un écart". Interrogé par l\'AFP, Matignon a précisé que les participants à la soirée étaient soumis à la présentation du pass sanitaire.\xa0 Dans la vidéo, publiée par un journaliste de La Voix du Nord, on voit Jean\xa0Castex et Gérald\xa0Darmanin serrer les mains des élus des Hauts-de-France reçus au ministère de l\'Intérieur en marge du congrès de l\'Association des maires de France (AMF) à Paris. Ce contenu est bloqué car vous n\'avez pas accepté les t

In [77]:
def fetch_summaries(li_of_links):
    d = defaultdict()
    for i in li_of_links:
        try:
            page_single = requests.get(i).text
            soup_single = BeautifulSoup(page_single)
            meta_tag = soup_single.find('meta', attrs={'name':'Description'})
            summary = meta_tag['content']
            d[i]=summary
        except:
            print(i)
            pass
    return dict(d)

In [145]:
def clean_text(row):
    search='Ce contenu est bloqué car vous n\'avez pas accepté les traceurs. En cliquant sur « J’accepte », les traceurs seront déposés et vous pourrez visualiser les contenus . En cliquant sur « J’accepte tous les traceurs », vous autorisez des dépôts de traceurs pour le stockage de vos données sur nos sites et applications à des fins de personnalisation et de ciblage publicitaire.'
    if search in str(row):
        print('yes')
        spl = row.split(search)
        new_row = ' '.join(spl)
        return new_row
    return row

In [159]:
def delete_cas(row):
    search = 'CAS N°1 : Vous (ou une autre personne) utilisez plus de $this.View.AuthResponse.DeviceLimit appareils et/ou navigateurs en même temps →\r\n\t\t\t\t\t\t\t\tDéconnectez-vous des appareils et/ou navigateurs que vous n\'utilisez pas CAS N°2 : Vous naviguez en mode privé →\r\n\t\t\t\t\t\t\t\tDéconnectez-vous systématiquement avant de fermer la fenêtre du navigateur CAS N°3 : Vous refusez les cookies de connexion dans les paramètres de votre navigateur (ou une mise à jour a modifié vos paramètres) →\r\n\t\t\t\t\t\t\t\tChanger les paramètres d\'acceptation des cookies de votre navigateur DANS TOUS LES CAS → cliquer sur "continuer sur cet appareil" résout le problème Que se passe-t-il si je clique sur "continuer sur cet appareil" ?\r\n\t\t\t\t\t\t\t\tVous pourrez profiter de votre compte sur cet appareil et tous vos autres appareils\r\n\t\t\t\t\t\t\t\tseront déconnectés. Vous pourrez toujours vous y reconnecter, dans la limite de\r\n\t\t\t\t\t\t\t\t$this.View.AuthResponse.DeviceLimit appareils. Comment puis-je voir les appareils connectés ?\r\n\t\t\t\t\t\t\t\tRendez-vous dans votre espace client puis cliquez sur "gérer les équipements". Si vous y allez après avoir cliqué sur "Continuer sur cet appareil", il ne devrait y en avoir qu\'un seul. Retrouvez tous nos contenus et notre journal en numériqueTéléchargez gratuitement l\'app'
    if search in str(row):
        return row[len(search):]
    return row

In [7]:
def list_of_dic(source):
    list_of_dic = []
    with open(source, "r") as a_file:
        for idx,line in enumerate(a_file):
            try:
                stripped_line = line.strip()
                dic=grab_info(stripped_line)
                print(idx)
                if dic['text']:
                    list_of_dic.append(dic)
            except ConnectionError:
                continue
    return list_of_dic

In [47]:
def write_csv(li_of_li, file_name):
    """this function writes a csv file
    from a list of lists"""
    with open(file_name, "wb") as f:
        writer = csv.writer(f)
        writer.writerows(li_of_li)

In [85]:
#ensure_ascii=False makes sure that french special characters are read correctly

with open('lest_republicain_summaries.json', 'w') as f:
    json.dump(li, f, ensure_ascii=False)

In [86]:
#converting a json to a csv file for convenience

data_file = open('csv_les_republicain_summ.csv', 'w', newline='')
csv_writer = csv.writer(data_file)

with open('lest_republicain_summaries.json') as json_file:
    jsondata = json.load(json_file)

    
count = 0
for data in jsondata:
    if count == 0:
        header = data.keys()
        csv_writer.writerow(header)
        count += 1
    csv_writer.writerow(data.values())

data_file.close()

In [ ]:
#use a set or hashmap

In [4]:
def update_csv(source_link):
    
    links_file = open('list_of_links-Copy1.txt', 'r')
    reader = csv.reader(links_file)
    allRows = [row[0] for row in reader]
    
    li_of_links = []
    
    for i in range(101):
        find = False

        url = source_link + '?page=' + str(i)
        page = requests.get(url)    
        data = page.text
        soup = BeautifulSoup(data)


        #catching bugs if a link doesn't contain a substring
        for link in soup.find_all('a'):

            if link.get('href')and link.get('href').startswith('/sante/2'):
                link_full = 'https://www.estrepublicain.fr'+link.get('href')
                if link_full in allRows and link_full != 'https://www.estrepublicain.fr/sante/2020/09/21/coronavirus-suivez-l-evolution-du-taux-d-incidence-dans-le-grand-est-et-en-bourgogne-franche-comte':
                    print(link_full)
                    find = True
                li_of_links.append(link_full+'\n')


                dic = grab_info(link_full)
                headersCSV = dic.keys()
                with open('csv_les_republicain-Copy1.csv', 'a', newline='') as f_object:
                    # Pass the CSV  file object to the Dictwriter() function
                    # Result - a DictWriter object
                    dictwriter_object = DictWriter(f_object, fieldnames=headersCSV)
                    # Pass the data in the dictionary as an argument into the writerow() function
                    dictwriter_object.writerow(dic)
                    # Close the file object
                    f_object.close()

        if find:
            break


    with open('list_of_links-Copy1.txt','a') as out:
        out.writelines(li_of_links)

In [42]:
def get_new(source='https://www.estrepublicain.fr/sante/coronavirus'):
    url = source
    page = requests.get(url)    
    data = page.text
    soup = BeautifulSoup(data)


    #catching bugs if a link doesn't contain a substring
    li_of_dict = []
    for link in soup.find_all('a'):
        date_today = str(datetime.today().strftime('%Y/%m/%d'))
        if link.get('href') and link.get('href').startswith('/sante/2'):
            link_full = 'https://www.estrepublicain.fr'+link.get('href')
            date_from_link = link_full.split('/sante/')[1].split('/')
            date_fin = '/'.join(date_from_link[:3])
            if date_fin == date_today:
                info = grab_info(link_full)
                if info:
                    li_of_dict.append(info)
    keys = list(li_of_dict[0].keys())

    a_file = open("lest_rep_today.csv", "w")
    dict_writer = csv.DictWriter(a_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(li_of_dict)
    a_file.close()

In [43]:
get_new(source='https://www.estrepublicain.fr/sante/coronavirus')

In [44]:
data = pd.read_csv("lest_rep_today.csv")

In [45]:
data

,date,summary,text,title,topic,url
0,2021/11/18,Suivez avec nous l'évolution de la pandémie de...,Un couple de Roubaisiens va être jugé en décem...,En direct. Covid-19 : pas de décision sur la v...,NaN,https://www.estrepublicain.fr/sante/2021/11/18...
1,2021/11/18,L’établissement est fermé au public depuis lun...,"À l’origine, « une salariée asymptomatique, po...",Giromagny. 18 cas de Covid à l’Ehpad Saint-Jos...,NaN,https://www.estrepublicain.fr/sante/2021/11/18...
2,2021/11/18,Selon les dernières données publiées par Santé...,Une situation épidémique similaire aux autres ...,Meuse. Covid-19 : le taux d’incidence est en h...,NaN,https://www.estrepublicain.fr/sante/2021/11/18...
3,2021/11/18,Lunéville renoue avec un centre de vaccination...,"A compter du lundi 22 novembre, retour dans la...",Lunéville. 3e déménagement pour le centre de v...,NaN,https://www.estrepublicain.fr/sante/2021/11/18...
